In [1]:
%run preamble.ipynb

The preamble handles a few imports and defines the following functions:

printLoss(y_tr,p_tr,y_te,p_te,loss=log_loss)
createDataSet(predictions, group_encoder, device_ids)
getBestPrediction(data,var='device_id')


A few Debug functions


In [2]:
ga_data = pd.read_csv("files/finalSets/ga_noEvts.csv"
                      , usecols=["device_id","group"
                                , "phone_brand","isTrain"
                                , "device_model"])

## Encode the  colums 

In [3]:
enc_brand = LabelEncoder()
enc_device = LabelEncoder()
enc_group = LabelEncoder()

In [4]:
ga_data["phone_brand"] = enc_brand.fit_transform(ga_data.phone_brand)
ga_data["device_model"] = enc_device.fit_transform(ga_data.device_model)
ga_data["group"] = enc_group.fit_transform(ga_data.group)

In [5]:
#from sklearn.preprocessing import OneHotEncoder
grd_enc = OneHotEncoder()
grd_enc.fit(ga_data.drop(["isTrain","group","device_id"],axis=1))

OneHotEncoder(categorical_features='all', dtype=<class 'float'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [6]:
drop_list = ["isTrain"]
train = ga_data[ga_data.isTrain==1].drop( drop_list, axis=1)
true_classes = train.group
test = ga_data[ga_data.isTrain==0].drop( drop_list+["group"], axis=1)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(train
                                                   , true_classes, test_size=0.3)

In [8]:

train_drop = ["device_id","group"]

## We use the best single model from previous tests
XGB classifier 

In [14]:
# the parameters have been determined in the `predict_classe` notebook.
gbdt = GradientBoostingClassifier(loss='deviance'
                                  ,max_features=None
                                  , min_samples_leaf=800
                                  , learning_rate=0.005
                                  , n_estimators=700);

In [48]:
gbdt.fit(x_train, y_train)

GradientBoostingClassifier(init=None, learning_rate=0.005, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=800, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=700,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [52]:
probs_train = gbdt.predict_proba(x_train)
probs_test = gbdt.predict_proba(x_test)

In [54]:
print("Test MVA predictions on test and training set:\n")
print("Log loss on training set: ", log_loss(y_train,probs_train))
print("Log loss on test set: ", log_loss(y_test,probs_test))

Test MVA predictions on test and training set:

Log loss on training set:  2.39388310435
Log loss on test set:  2.40801068014


Performs largely identical.

## compare with linear model

In [196]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l1'
                        , C=0.6
                        , tol=0.000001
                        , solver= "liblinear"#"newton-cg"##'lbfgs'
                        , max_iter=2000
                        #, fit_intercept = False
                        #, class_weight = 'balanced'
                        #, multi_class='multinomial'
                       #,warm_start=True
                       )

In [197]:
lr.fit(grd_enc.fit_transform(x_train.drop(train_drop,axis=1)),y_train)

LogisticRegression(C=0.6, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=2000, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=1e-06,
          verbose=0, warm_start=False)

In [198]:
probs_lr_train = lr.predict_proba(grd_enc.transform(x_train.drop(train_drop,axis=1)))
probs_lr_test = lr.predict_proba(grd_enc.transform(x_test.drop(train_drop,axis=1)))

In [199]:
print("Test log-regression predictions on test and training set:\n")
print("Log loss on training set: ", log_loss(y_train,probs_lr_train))
print("Log loss on test set: ", log_loss(y_test,probs_lr_test))

Test log-regression predictions on test and training set:

Log loss on training set:  2.34649096859
Log loss on test set:  2.39829927098


## Try linear model ontop of gbdt

In [35]:
gbdt = joblib.load('trainedModels/final_gbdt_goNoEvts.pkl')

In [37]:
x_tr_lr, x_te_lr, y_tr_lr, y_te_lr = train_test_split(x_test, y_test, test_size=0.5)

In [38]:
grd_enc.fit(gbdt.apply(x_train)[:,:,0])
lr.fit(grd_enc.transform(gbdt.apply(x_tr_lr)[:,:,0]), y_tr_lr)

LogisticRegression(C=0.05, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=200, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [39]:
prob_gbdt_lr = lr.predict_proba(grd_enc.transform(gbdt.apply(x_te_lr)[:,:,0]))

In [40]:
prob_gbdt_lr_train = lr.predict_proba(grd_enc.transform(gbdt.apply(x_train)[:,:,0]))

In [41]:
print("Test log-regression predictions on test and training set:\n")
print("Log loss on training set: ", log_loss(y_train,prob_gbdt_lr_train))
print("Log loss on test set: ", log_loss(y_te_lr,prob_gbdt_lr))

Test log-regression predictions on test and training set:

Log loss on training set:  2.43667749883
Log loss on test set:  2.43049368416


# Save final xgb classifier

In [81]:
joblib.dump(gbdt, 'trainedModels/final_gbdt_goNoEvts.pkl', compress=3)

['trainedModels/final_gbdt_goNoEvts.pkl']

# Predict test sample and create submission file (part of it)

In [83]:
predictions = gbdt.predict_proba(test.drop(["device_id"], axis=1))

In [85]:
probas_gbdt_df = pd.DataFrame(predictions, index=test.device_id)
probas_gbdt_df.columns = enc_group.inverse_transform(probas_gbdt_df.columns)

In [88]:
probas_gbdt_df.to_csv("finalOutputs/gbdt_gaNoEvts_predictions.csv")

### can also use linear output


In [201]:
preds = lr.predict_proba(grd_enc.transform(test.drop("device_id",axis=1)))

In [202]:
probas_lr_df = pd.DataFrame(preds, index=test.device_id)
probas_lr_df.columns = enc_group.inverse_transform(probas_lr_df.columns)
probas_lr_df.to_csv("finalOutputs/lr_gaNoEvts_prediction.csv")

# Random forest

In [1]:
from sklearn.ensemble import RandomForestClassifier

In [50]:
rf = RandomForestClassifier(n_estimators=1000
                            , criterion='entropy'
                            #, max_depth=20
                            , min_samples_split=2500
                            , min_samples_leaf=1
                            , max_leaf_nodes=None
                            , n_jobs=3
                            , random_state=666
                            , verbose=1
                            #, warm_start=True
                           )

In [51]:
s=time.time()/60.0
rf.fit(grd_enc.transform(x_train.drop(train_drop,axis=1)),y_train)
print("rf done in ", (time.time()/60.0)-s, " minutes.")

[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    2.7s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:   12.0s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:   26.7s
[Parallel(n_jobs=3)]: Done 794 tasks      | elapsed:   48.7s


rf done in  1.0360571630299091  minutes.


[Parallel(n_jobs=3)]: Done 1000 out of 1000 | elapsed:  1.0min finished


In [52]:
probs_rf_train = rf.predict_proba(grd_enc.transform(x_train.drop(train_drop,axis=1)))
probs_rf_val = rf.predict_proba(grd_enc.transform(x_test.drop(train_drop,axis=1)))
printLoss(y_train, probs_rf_train, y_test, probs_rf_val)

[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.1s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.9s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:    2.1s
[Parallel(n_jobs=3)]: Done 794 tasks      | elapsed:    4.1s
[Parallel(n_jobs=3)]: Done 1000 out of 1000 | elapsed:    5.2s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.3s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:    0.8s
[Parallel(n_jobs=3)]: Done 794 tasks      | elapsed:    1.6s
[Parallel(n_jobs=3)]: Done 1000 out of 1000 | elapsed:    2.0s finished


Test MVA predictions on test and training set:

Log loss on training set:  2.34558622624
Log loss on test set:  2.39882036897


# BDT

In [16]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import AdaBoostClassifier

In [29]:
# no hyper-parameter tuning, just from experience...
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=9),
                         #algorithm="SAMME.R",
                         learning_rate=0.1,
                         n_estimators=1200)

In [30]:
s = time.time()
bdt.fit(x_train.drop(train_drop, axis=1), y_train)
print("bdt trained in ", (time.time()-s)/60.0, " minutes")

bdt trained in  1.840395983060201  minutes


In [31]:
#probs_bdt_train = bdt.predict_proba(grd_enc.transform(x_train.drop(train_drop,axis=1)))
#probs_bdt_val = bdt.predict_proba(grd_enc.transform(x_test.drop(train_drop,axis=1)))


probs_bdt_train = bdt.predict_proba(x_train.drop(train_drop,axis=1))
probs_bdt_val = bdt.predict_proba(x_test.drop(train_drop,axis=1))
printLoss(y_train, probs_bdt_train, y_test, probs_bdt_val)

Test MVA predictions on test and training set:

Log loss on training set:  2.38713265857
Log loss on test set:  2.5226521017
